In [2]:
import os
import pickle
from scipy import signal
import libfmp.b
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from load_djembe_marker import *
from foot_module import onset_calculations, onset_extraction, onset_filtering, utils, onset_plot
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, argrelmin, argrelmax
from pydub import AudioSegment
from scipy.signal import find_peaks, peak_prominences
from pydub.generators import Triangle

import mir_eval
from compute_tempo import *
from dance_evaluation import *
from mutils import DataHandler

OnsetProcessor = onset_calculations.OnsetProcessor()

In [ ]:
mvnfiles = ["BKO_E1_D1_01_Suku_T.mvnx", "BKO_E1_D1_02_Maraka_T.mvnx", "BKO_E1_D1_03_Wasulunka_T.mvnx", "BKO_E1_D2_04_Maraka_T.mvnx"]

filename = mvnfiles[1]
piece_name = os.path.basename(filename).split(".")[0]
data_handler = DataHandler()
motion_data, drum_onsets, start_f, end_f, start_t, end_t, cycle_onsets, beat_ref, bpm = data_handler.load_and_process_data(filename, mode = 'gr', drum = 'J2', section_idx=0)


sensorA_velocity = motion_data['velocity']["SEGMENT_LEFT_FOOT"]
sensorA_velocity_znorm = z_score_normalize(sensorA_velocity)    # normalize -1 to 1 to retain direction

 # keep positive directions
sensor_abs_vel = smooth_velocity(sensorA_velocity_znorm, abs="yes", window_length = 60, polyorder = 0 ) # size (n, 3)
# sensor_abs_vel[sensor_abs_vel < 0] = 0       
sensorA_velocity_mmnorm = min_max_normalize(sensor_abs_vel)     # normalize between 0-1

ax = 2
sensorA_velocity_ax = sensorA_velocity_mmnorm[start_f:end_f, ax]

#####################
peaks_temp, _ = find_peaks(sensorA_velocity_ax.flatten(), height= 0.02, distance=60)
peak_values = sensorA_velocity_ax[peaks_temp]
h_cutoff = np.percentile(peak_values, 40)
print(h_cutoff)

peaks, _ = find_peaks(sensorA_velocity_ax.flatten(), height= h_cutoff, prominence= h_cutoff)
peak_onset = np.zeros(len(sensorA_velocity_ax))
peak_onset[peaks] = 0.2
######################+


plt.figure(figsize=(40, 6), dpi=100)
plt.plot(sensorA_velocity_ax, linewidth=1, color = 'b')

plt.plot(peak_onset, linewidth=1, color = 'r')


In [11]:
pkl_filelist = os.listdir(f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/Dataset_V2")
data_handler = DataHandler()

mode_list = ["zero_uni", "zero_bi", "peak_uni", "peak_bi", "zero_bothfeet"]

for mode in mode_list:
    result = {
    "piece": [],
    "bpm": [],
    "tempoA": [],
    "tempoB": [],
    }
    for filename in pkl_filelist:

        filename = filename.replace("_Dancers.csv", "_T.mvnx")
        piece_name = os.path.basename(filename).split(".")[0]
        try:
            motion_data, drum_onsets, start_f, end_f, start_t, end_t, cycle_onsets, beat_ref, bpm, feet_onsets = data_handler.load_and_process_feetdata(filename, mode = 'gr', drum = 'J2', section_idx=0)
        except ValueError as e:
            # Handle the error and continue
            print(f"Error encountered for: {e}")
            continue
        
        novelty_length = len(feet_onsets)
        time_axis = np.arange(novelty_length)/240

        duration = int(end_t-start_t)
        w_sec = int(duration)
        h_sec = int(w_sec/5)
        
        mocap_fps = 240
        window_size = int(240*w_sec)
        hop_size = int(240*h_sec)
        tempi_range = np.arange(70,145,1)
        
        sensorA_velocity = motion_data['velocity']["SEGMENT_LEFT_FOOT"][start_f:end_f, :] 
        sensorA_pos = motion_data['position']["SEGMENT_LEFT_FOOT"][start_f:end_f, :]
        
        sensorA_velocity_norm = z_score_normalize(sensorA_velocity)
        sensorA_pos_norm = z_score_normalize(sensorA_pos)
        sensorA_positionde = detrend_signal_array(sensorA_pos_norm, cutoff= 0.5)
        sensorA_positionde = smooth_velocity(sensorA_positionde, abs="no") # size (n, 3)

        # mode: zero_uni zero_bi peak_uni   peak_bi     zero_bothfeet
        # mode = "zero"
        tempo_json_two_sensor = main_two_sensor_feet(sensorA_velocity_norm, sensorA_positionde, feet_onsets, 
                                                    mocap_fps, window_size,hop_size, tempi_range, mode= mode, ax=2)

        # feet_sensor_onsets = tempo_json_two_sensor["feet_sensor_onsets"]
        # tempogram_ab = tempo_json_two_sensor["tempogram_ab"]
        # tempogram_raw = tempo_json_two_sensor["tempogram_raw"]
        # time_axis_seconds = tempo_json_two_sensor["time_axis_seconds"]
        # tempo_axis_bpm = tempo_json_two_sensor["tempo_axis_bpm"]

        tempo_data_maxmethod = tempo_json_two_sensor["tempo_data_maxmethod"]
        tempo_data_weightedkernel = tempo_json_two_sensor["tempo_data_weightedkernel"]
        # tempo_data_combinedtempogram = tempo_json_two_sensor["tempo_data_combinedtempogram"]

        # Max method
        Aestimated_beat_pulse = tempo_data_maxmethod["estimated_beat_pulse"]
        Atempo_curve = tempo_data_maxmethod["tempo_curve"]
        # Atempo_curve_time_axis = tempo_data_maxmethod["tempo_curve_time_axis"]
        # Aglobal_tempo_bpm = tempo_data_maxmethod["global_tempo_bpm"]

        # Weighted method
        Bestimated_beat_pulse = tempo_data_weightedkernel["estimated_beat_pulse"]
        Btempo_curve = tempo_data_weightedkernel["tempo_curve"]
        # Btempo_curve_time_axis = tempo_data_weightedkernel["tempo_curve_time_axis"]
        # Bglobal_tempo_bpm = tempo_data_weightedkernel["global_tempo_bpm"]
        
        tempo_A = np.round(np.average(Atempo_curve), 2)  # Average over the desired frame range
        tempo_B = np.round(np.average(Btempo_curve), 2)

        result["piece"].append(piece_name)
        result["bpm"].append(np.round(bpm,2))
        result["tempoA"].append(np.round(tempo_A,2))
        result["tempoB"].append(np.round(tempo_B,2))    

    results_df = pd.DataFrame(result)
    csv_filename = f"./results/log01/foot/Z/foot_z_{mode}_50_150.csv"
    results_df.to_csv(csv_filename, index=False)
    print(f"Results saved to {csv_filename}")

Loaded BKO_E1_D1_08_Suku_T.pkl
Error encountered for: Group 'gr' not found in the dataset.
Loaded BKO_E3_D5_03_Wasulunka_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 114.20 BPM
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E2_D4_05_Sandia_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 91.98 BPM
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E2_D4_01_Suku_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 128.84 BPM
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E2_D4_03_Wasulunka_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 108.26 BPM
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E1_D2_04_Maraka_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section:

/itf-fi-ml/home/sagardu/dance_tempo_estimation/compute_tempo.py:834: RuntimeWarning: invalid value encountered in scalar divide
  selected_bpm = np.sum(bpm_arr * magnitude_arr) / np.sum(magnitude_arr)


Computing max method...
Computing weighted method...
Loaded BKO_E1_D2_04_Maraka_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 101.71 BPM
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E3_D5_01_Maraka_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 113.68 BPM
Computing tempograms...
Computing max method...
Computing weighted method...
Loaded BKO_E3_D6_02_Suku_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 136.13 BPM
Computing tempograms...


/itf-fi-ml/home/sagardu/dance_tempo_estimation/compute_tempo.py:834: RuntimeWarning: invalid value encountered in scalar divide
  selected_bpm = np.sum(bpm_arr * magnitude_arr) / np.sum(magnitude_arr)


Computing max method...
Computing weighted method...
Loaded BKO_E2_D4_06_Manjanin_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 129.54 BPM
Computing tempograms...


...

/itf-fi-ml/home/sagardu/dance_tempo_estimation/compute_tempo.py:834: RuntimeWarning: invalid value encountered in scalar divide
  selected_bpm = np.sum(bpm_arr * magnitude_arr) / np.sum(magnitude_arr)


Loaded BKO_E3_D5_02_Suku_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 127.11 BPM
Computing tempograms...
Computing max method...
Computing weighted method...


/itf-fi-ml/home/sagardu/dance_tempo_estimation/compute_tempo.py:834: RuntimeWarning: invalid value encountered in scalar divide
  selected_bpm = np.sum(bpm_arr * magnitude_arr) / np.sum(magnitude_arr)


Loaded BKO_E3_D6_12_Suku_T.pkl
Total Sections: 1
index 0 for first section
J2 tempo for the current section: 141.70 BPM
Computing tempograms...
Computing max method...
Computing weighted method...
Results saved to ./results/log01/foot/Z/foot_z_zero_bothfeet_50_150.csv


/itf-fi-ml/home/sagardu/dance_tempo_estimation/compute_tempo.py:834: RuntimeWarning: invalid value encountered in scalar divide
  selected_bpm = np.sum(bpm_arr * magnitude_arr) / np.sum(magnitude_arr)


In [ ]:
nn = 3
mvnfiles = ["BKO_E1_D1_01_Suku_T.mvnx", "BKO_E1_D1_02_Maraka_T.mvnx", "BKO_E1_D1_03_Wasulunka_T.mvnx", "BKO_E1_D2_04_Maraka_T.mvnx"]
# annotation = ["BKO_E1_D1_01_Suku_Dancers.csv", "BKO_E1_D1_02_Maraka_Dancers.csv", "BKO_E1_D1_03_Wasulunka_Dancers.csv", "BKO_E1_D2_04_Maraka.csv"]
# cyclefile= ["BKO_E1_D1_01_Suku_C.csv", "BKO_E1_D1_02_Maraka_C.csv", "BKO_E1_D1_03_Wasulunka_C.csv", "BKO_E1_D2_04_Maraka_C.csv"]


file_path = f"./mocap_mvnx/{mvnfiles[nn]}"
file_name = os.path.basename(file_path).split(".")[0]

pickle_path = f'./output/motion_data_pkl/{file_name}.pkl'
    
if os.path.isfile(pickle_path):
    with open(pickle_path, 'rb') as file:
        motion_data = pickle.load(file)
    print(f"Loaded {file_name}.pkl")

onset_filename = file_name.replace("_T", "")
danceanno_filename = file_name.replace("_T", "_Dancers")
mcycle_filename = file_name.replace("_T", "_C")

drum_onsets_path = f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/drum_onsets/{onset_filename}.csv"
dance_anno_path = f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/dance_annotations_all/{danceanno_filename}.csv"
mcycle_path = f"/itf-fi-ml/home/sagardu/djembe_drive/sgr_pyspace/metric_cycles/{mcycle_filename}.csv"

# Load meter cycle onsets
df_metric = pd.read_csv(mcycle_path)
loaded_mcycle_onsets = np.round(df_metric["Selected Onset"].to_numpy(), 3)

feet_length = len(motion_data['position']['SEGMENT_RIGHT_FOOT']) # size (n, 3)
feet_onsets_path = f"./logs/{file_name}/onset_info/{file_name}_both_feet_onsets.csv"
feet_df = pd.read_csv(feet_onsets_path, usecols=[1])
feet_frames = feet_df.values.flatten()

feet_onsets = np.zeros(feet_length)
feet_onsets[feet_frames] = 1
feet_onsets = np.reshape(feet_onsets, (-1, 1))


In [ ]:
drum_df = pd.read_csv(drum_onsets_path)
column_b = drum_df["J2"].dropna()
drum_onsets = column_b.to_numpy()

anno_df = pd.read_csv(dance_anno_path)
category_df = anno_df.groupby('mocap')
category_df = category_df.get_group('gr')
category_df = category_df.reset_index(drop=True)
category_df

section_data = OnsetProcessor.onset_calculations(category_df, loaded_mcycle_onsets)
section_idx = 0  # Replace with the index you need from the dance_annotation_df table

section = section_data[section_idx]
section_name, section_info = list(section.items())[0]

section_onset_data = section_info["section_onset_data"]
cycle_onsets = section_onset_data["cycle_onsets"]

In [ ]:
cycle_t = cycle_onsets
# Initialize an empty list to store all beat onsets
beat_ref = []

# Iterate through each pair of consecutive cycle onsets
for i in range(len(cycle_t) - 1):
    start = cycle_t[i]
    end = cycle_t[i + 1]
    
    # Generate beat onsets by dividing the interval into 4 equal parts
    beat_onsets = np.linspace(start, end, num=5)[:-1]  # 4 intervals, excluding the endpoint
    beat_ref.extend(beat_onsets)

# Convert the result to a numpy array
beat_ref = np.array(np.round(beat_ref,3))


In [ ]:
# Calculate tempo in beats per minute (BPM)

iois = np.diff(beat_ref)
mean_ioi = np.mean(iois)
bpm = 60 / mean_ioi
print(f"Tempo: {bpm:.2f} BPM")

In [ ]:
row = 0
start_f = np.round(category_df.iloc[row, 6]*240).astype(int)
end_f = np.round(category_df.iloc[row, 7]*240).astype(int)
start_t = start_f/240
end_t = end_f/240

start_t

### From extracted feet onsets

In [ ]:
novelty_length = len(feet_onsets)
time_axis = np.arange(novelty_length)/240

mocap_fps = 240
window_size = 240*20 #corresponding to 10 sec
hop_size = 240*10
tempi_range = np.arange(59,181,1)

tempo_json_two_sensor = main_two_sensor_feet(feet_onsets, mocap_fps, window_size, hop_size, tempi_range)

feet_sensor_onsets = tempo_json_two_sensor["feet_sensor_onsets"]
tempogram_ab = tempo_json_two_sensor["tempogram_ab"]
tempogram_raw = tempo_json_two_sensor["tempogram_raw"]
time_axis_seconds = tempo_json_two_sensor["time_axis_seconds"]
tempo_axis_bpm = tempo_json_two_sensor["tempo_axis_bpm"]

tempo_data_maxmethod = tempo_json_two_sensor["tempo_data_maxmethod"]
tempo_data_weightedkernel = tempo_json_two_sensor["tempo_data_weightedkernel"]
# tempo_data_combinedtempogram = tempo_json_two_sensor["tempo_data_combinedtempogram"]

# Max method
Aestimated_beat_pulse = tempo_data_maxmethod["estimated_beat_pulse"]
Atempo_curve = tempo_data_maxmethod["tempo_curve"]
Atempo_curve_time_axis = tempo_data_maxmethod["tempo_curve_time_axis"]
Aglobal_tempo_bpm = tempo_data_maxmethod["global_tempo_bpm"]

# Weighted method
Bestimated_beat_pulse = tempo_data_weightedkernel["estimated_beat_pulse"]
Btempo_curve = tempo_data_weightedkernel["tempo_curve"]
Btempo_curve_time_axis = tempo_data_weightedkernel["tempo_curve_time_axis"]
Bglobal_tempo_bpm = tempo_data_weightedkernel["global_tempo_bpm"]


In [ ]:
plt.figure(figsize=(40, 6), dpi=200)
plt.plot(100*Aestimated_beat_pulse[start_f:end_f], linewidth=1, color = 'b')
plt.plot(Atempo_curve[start_f:end_f], linewidth=1, color = 'r')


plt.xlabel('Time (seconds)')
plt.ylabel('Tempo (BPM)')
plt.title(f'{file_name} Start:{round(start_t)} End:{round(end_t)}')
plt.grid(True)
plt.show()

In [ ]:
# Tempogram

fig, axs = plt.subplots(1, 1, figsize=(5,5), dpi=100)
cax1 = axs.pcolormesh(time_axis_seconds, tempo_axis_bpm, tempogram_ab[0], shading='auto', cmap='magma')
axs.set_title('Z-axis')
axs.set_xlabel('Time [s]')
axs.set_ylabel('Tempo [BPM]')
plt.colorbar(cax1, ax=axs, orientation='horizontal', label='Magnitude')
plt.show()

In [ ]:
# drum onset and directional change onset plot

plt.figure(figsize=(40, 6), dpi=300)
plt.plot(feet_sensor_onsets, linewidth=0.3, color = 'red')
# plt.plot(Atempo_curve_time_axis, Atempo_curve, linewidth=0.3, color = 'red')
plt.vlines(x=np.round(drum_onsets*240), ymin=0.0, ymax=0.5, color='b', linestyle='dotted', linewidth=0.3)
plt.vlines(x=np.array([start_f, end_f]), ymin=0.0, ymax=1, color='g', linewidth=1)


plt.xlabel('Time (seconds)')
plt.ylabel('Tempo (BPM)')
plt.title('Beat Pulse')
plt.grid(True)
plt.show()

In [ ]:
# Per mode: drum onset and directional change onset plot

drum_ref = drum_onsets[(drum_onsets >= start_t) & (drum_onsets <= end_t)]
half_beats = (drum_ref[:-1] + drum_ref[1:]) / 2
drum_subref = np.sort(np.concatenate((drum_ref, half_beats)))
# raw onsets
onsets_frm = np.where(feet_sensor_onsets > 0)[0]     # Frames idx of onsets
onsets_t = onsets_frm/240
dance_onset = onsets_t[(onsets_t >= start_t) & (onsets_t <= end_t)]

# estimated beats
time = np.arange(novelty_length) / mocap_fps
peaks, _ = signal.find_peaks(Bestimated_beat_pulse)  # , prominence=0.02
peaks_t = time[peaks]
beat_estimated = peaks_t[(peaks_t >= start_t) & (peaks_t <= end_t)]

# print("ref max ioi:", max(np.diff(drum_ref)))
# print("ref min ioi:", min(np.diff(drum_ref)))
# print("ref average ioi:", np.average(np.diff(drum_ref)))
# Plot figure
plt.figure(figsize=(40, 6), dpi=300)

window_size = 0.1
for onset in beat_ref:
    window_start = onset - (window_size/2)  # Start of the window (25ms before)
    window_end = onset + (window_size/2)   # End of the window (25ms after)
    
    # Plot shaded window
    plt.axvspan(window_start, window_end, color='red', alpha=0.3)
    # Plot reference onset as a vertical line
    plt.axvline(onset, color='red', linestyle='--', linewidth=0.9)

plt.vlines(x= dance_onset, ymin=0.0, ymax=1, color='g', linewidth=0.5,)

plt.xlabel('Time (seconds)')
plt.title(f'{file_name}')
plt.grid(True)
plt.show()

In [ ]:
results = evaluate_dance_onsets_with_half_beats(beat_ref, dance_onset, tolerance=0.2)
print("\nEvaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")

In [ ]:
# Evaluate using mir_eval
# A significant increase in Recall with a slight decrease in Precision 
# may indicate that dancers are indeed synchronizing with subdivisions.


half_beats = (drum_ref[:-1] + drum_ref[1:]) / 2
combined_ref = np.sort(np.concatenate((drum_ref, half_beats)))

# scores = mir_eval.beat.evaluate(drum_reference, beat_estimated)
print("drum reference | ")
precision, recall, f_measure = mir_eval.onset.f_measure(drum_ref, beat_estimated, window= 0.2)
print(f'Precision: {precision}, Recall: {recall}, F-Measure: {f_measure}')

print("combined half-beat | ")
precision, recall, f_measure = mir_eval.onset.f_measure(combined_ref, beat_estimated, window= 0.2)
print(f'Precision: {precision}, Recall: {recall}, F-Measure: {f_measure}')

# dance onsets are less than reference onsets and thus the evaluation gives poor results.
# if dance onsets are not present for reference onsets than it should not be penalized.

In [ ]:
onsets_val = drum_ref         # Onset values in seconds
window_size = 0.1  # 50ms window (±25ms)

# Plotting
plt.figure(figsize=(10, 2), dpi=300)
plt.vlines(x= dance_onset, ymin=0.0, ymax=1, color='b', linewidth=0.5,)
# Iterate through onsets and plot vertical lines and windows
for onset in onsets_val:
    window_start = onset - (window_size/2)  # Start of the window (25ms before)
    window_end = onset + (window_size/2)   # End of the window (25ms after)
    
    # Plot shaded window
    plt.axvspan(window_start, window_end, color='green', alpha=0.3)
    # Plot reference onset as a vertical line
    plt.axvline(onset, color='red', linestyle='--', linewidth=0.5)

# Customize the plot
plt.title("Visualization of Onsets with 50ms Windows")
plt.xlabel("Time (s)")
plt.xlim([min(onsets_val), max(onsets_val) + 1])  # Extend x-axis slightly beyond the last onset
plt.ylim([0, 1])  # Arbitrary y-axis for visualization
plt.yticks([])  # Hide y-axis ticks
plt.tight_layout()
plt.show()

In [ ]:

time = np.arange(novelty_length) / mocap_fps
peaks, properties = signal.find_peaks(Aestimated_beat_pulse)

# time = np.arange(end_f-start_f) / mocap_fps
# peaks, properties = signal.find_peaks(Aestimated_beat_pulse[start_f:end_f])  # , prominence=0.02

beat_peaks_sec = time[peaks]

click_duration = 50  # milliseconds
click_freq = 1200  # Hz
file_name ="suku"

# Generate a single click sound
click = Triangle(click_freq).to_audio_segment(duration=click_duration)

onset_times = beat_peaks_sec  # kept_onsets/240   beat_peaks_sec
dN = novelty_length
# dN = end_f-start_f
total_duration = (dN/240)*1000  #  in milliseconds

audio = AudioSegment.silent(duration=total_duration)
for onset in onset_times:
    position = int(onset * 1000)  # Convert onset time to milliseconds
    audio = audio.overlay(click, position=position)

# Export the audio with clicks to a file
audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Both_Foot_weighted.wav"), format="wav")
# audio.export(os.path.join("/itf-fi-ml/home/sagardu/extract_feet_onset", f"{file_name}_Bothhand_dir.wav"), format="wav")